In [1]:
# Importar Librerias
import pandas as pd
import pyodbc 

#Establecer la conexión con la base de datos
server = 'ws-dev' 
database = 'svs_produccion' 
username = 'sa_Api' 
password = '$db@C0vid*19' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

print("Conexion exitosa")

# Ejecutar una consulta SQL para obtener la lista de tablas
df = pd.read_sql_query("SELECT * FROM Rpt_ResultadosLaboratorio",cnxn)

#Cerrar Conexion a la Base de Datos
cnxn.close()

# validar los campos o valiables del dataframe
df.head()

Conexion exitosa


C:\Users\wsilva\AppData\Local\Temp\ipykernel_19044\174594420.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM Rpt_ResultadosLaboratorio",cnxn)


,OrdenLaboratorioDetalleId,FechaRecepcionMuestra,CodigoMuestra,Expediente,NombreCompleto,Edad,TipoEdad,Sexo,Ocupacion,Region,...,Obesidad,Asma,EnfermedadRenalCronica,Inmunosupresion,AlcoholismoCronico,EnfermedadNeurologicaCronica,Tabaquismo,Embarazo,SemanasGestacion,Observaciones
0,32382,2020-12-07 20:20:34.127,LNV 202490,0801198511570,JOSUE DAVID SOLANO CERRATO,35,AÑOS,HOMBRE,SD,19,...,None,None,None,None,None,None,None,None,NaN,
1,32380,2020-12-07 20:15:53.620,LNV 202488,0801202009731,MATEO SEBASTIAN SALVADOR AVILA,1,MESES,HOMBRE,SD,19,...,None,None,None,None,None,None,None,None,NaN,
2,25795,2020-12-01 13:26:44.477,LNV 195903,P-3700,JOSUE FERNANDO PASCUAL TORRES,31,AÑOS,HOMBRE,SD,19,...,None,None,None,None,None,None,None,None,NaN,
3,31965,2020-12-07 15:04:56.323,LNV 202073,0704199400390,JUSSARA SIBELLY AVILA RODRIGUEZ,27,AÑOS,MUJER,SD,19,...,None,None,None,None,None,None,None,None,NaN,
4,32216,2020-12-07 17:05:42.040,LNV 202324,0716199000118,JOSE MANUEL CACERES LOPEZ,30,AÑOS,HOMBRE,SD,19,...,None,None,None,None,None,None,None,None,NaN,


In [2]:
## importar librerias
import sidetable
import datetime

## crear nuevas columnas de mes y año en el data frame
df['num_mes'] = df['FechaResultado'].dt.month
df['mes'] = df['FechaResultado'].dt.strftime("%B")
df['año'] = df['FechaResultado'].dt.year

df.head()


,OrdenLaboratorioDetalleId,FechaRecepcionMuestra,CodigoMuestra,Expediente,NombreCompleto,Edad,TipoEdad,Sexo,Ocupacion,Region,...,Inmunosupresion,AlcoholismoCronico,EnfermedadNeurologicaCronica,Tabaquismo,Embarazo,SemanasGestacion,Observaciones,num_mes,mes,año
0,32382,2020-12-07 20:20:34.127,LNV 202490,0801198511570,JOSUE DAVID SOLANO CERRATO,35,AÑOS,HOMBRE,SD,19,...,None,None,None,None,None,NaN,,12,December,2020
1,32380,2020-12-07 20:15:53.620,LNV 202488,0801202009731,MATEO SEBASTIAN SALVADOR AVILA,1,MESES,HOMBRE,SD,19,...,None,None,None,None,None,NaN,,12,December,2020
2,25795,2020-12-01 13:26:44.477,LNV 195903,P-3700,JOSUE FERNANDO PASCUAL TORRES,31,AÑOS,HOMBRE,SD,19,...,None,None,None,None,None,NaN,,12,December,2020
3,31965,2020-12-07 15:04:56.323,LNV 202073,0704199400390,JUSSARA SIBELLY AVILA RODRIGUEZ,27,AÑOS,MUJER,SD,19,...,None,None,None,None,None,NaN,,12,December,2020
4,32216,2020-12-07 17:05:42.040,LNV 202324,0716199000118,JOSE MANUEL CACERES LOPEZ,30,AÑOS,HOMBRE,SD,19,...,None,None,None,None,None,NaN,,12,December,2020


In [380]:
df.stb.freq(['Resultado'], style=True)

,Resultado,count,percent,cumulative_count,cumulative_percent
0,NEGATIVO,"824,751",74.64%,"824,751",74.64%
1,POSITIVO,"278,283",25.18%,"1,103,034",99.82%
2,NO SE PROCESO,"1,331",0.12%,"1,104,365",99.95%
3,REPETIR,607,0.05%,"1,104,972",100.00%


In [3]:
## crear condiciones para filtrar datos para pruebas de COVID
condicion1 = df.Prueba.isin(["SARS CoV-2 (ANTIGENO-RDT)","SARS CoV-2 (RT-PCR)"])
condicion2 = df["año"]>2019

## ejecutar filtro de datos
df_covid = df[condicion1 & condicion2]

### Creacion de tabla de frecuencias absolutas y relativas de Casos x Año en Honduras

In [4]:
## Crear tabla  que muestre la tasa de incidencia de casos por año en Honduras
## Y Ademas muestre el % de positividad de casos acumulados por año
## tasa calculada por 100,000 habitantes

# paso1. crear una tabla resumen agrupando los casos por año
df_Totales = df_covid.groupby(['año','Resultado']).size().reset_index(name='casos')
df_Totales = df_Totales.pivot(index="año",columns="Resultado", values="casos")

# paso2. importar data set de la poblacion de Honduras por año
df_poblacion = pd.read_excel("../dataset/poblacionHN.xlsx")

# paso3. crear un join con el data frame de poblacion y 
# agregar un campo calculando la tasa * 100,000 habitantes
# agregar un campo calculando % Positividad
# inner join con tabla de poblacion
df_Totales = pd.merge(df_Totales,df_poblacion, on='año',how='inner')
df_Totales['Tasa_100mil'] = (round((df_Totales['POSITIVO'] / df_Totales['poblacion']) * 100000)).astype('int')
df_Totales['Total_Muestras'] = df_Totales['NEGATIVO']+df_Totales['POSITIVO']
df_Totales['Positividad%'] = (df_Totales['POSITIVO'] / df_Totales['Total_Muestras']).apply('{:.0%}'.format)
df_Totales['año'] = df_Totales['año'].astype('string')

# Seleccion de Columnas
df_Totales = df_Totales[["año","poblacion","NEGATIVO","POSITIVO","Total_Muestras","Positividad%","Tasa_100mil"]]
df_Totales.to_excel("../resultados/df_Totales.xlsx")
print(df_Totales)

    año  poblacion  NEGATIVO  POSITIVO  Total_Muestras Positividad%  \
0  2020    9304380     21651     11102           32753          34%   
1  2021    9450711    427223    182486          609709          30%   
2  2022    9597739    307826     81106          388932          21%   
3  2023    9745149     69278      3659           72937           5%   

   Tasa_100mil  
0          119  
1         1931  
2          845  
3           38  


### Grafico_1 Incidencia de Casos de COVID19 por Año en Honduras

In [7]:
## Crear grafico de barras con plotly  para visualizar la incidencia de casos por año en Honduras
## tasa calculada por 100,000 habitantes
## Tasa Incidencia = (Positivos / Poblacion) x 100 mil habitantes

import plotly.offline as pyo 
import plotly.express as px
import plotly.graph_objs as go 

data = [go.Bar(x=df_Totales["año"],y=df_Totales["Tasa_100mil"],marker={"color":"#2E7DA1"})]
layout = go.Layout(title="Incidencia de Casos de COVID19 por Año en Honduras", font=dict(size=18),
                   yaxis=dict(title="Tasa x 100,000 Habitantes"),
                   xaxis=dict(title="Años")                   
                   )

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig,filename="../resultados/Bar_Plot_IncidenciaByAnio.html")
### Codigo para libreria plotly express
# fig1 = px.bar(df_Totales, x="año", y="tasa", color="año")
# fig1.show()


'../resultados/Bar_Plot_IncidenciaByAnio.html'

### Grafico_2 % de Positividad Casos de COVID19 por Año en Honduras

In [310]:
## Crear grafico de barras con plotly  para visualizar el % de Positividad de casos acumulados por año en Honduras
## Positividad = Positivos / Total Muestras

df_Totales['Positividad%'] = df_Totales['POSITIVO'] / df_Totales['Total_Muestras']

data = [go.Bar(x=df_Totales["año"],y=df_Totales["Positividad%"],marker={"color":"#38BCB0"})]
layout = go.Layout(title="% Positividad COVID19 por Año en Honduras", font=dict(size=18),
                   yaxis=dict(title=" % de Positividad", tickformat=".0%"),
                   xaxis=dict(title="Años")                   
                   )

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig,filename="../resultados/Bar_Plot_PositividadByAnio.html")

'../resultados/Bar_Plot_PositividadByAnio.html'

### Tabla de Serie Temporal de %Positividad a Nivel Nacional
#### Grafico_3 Serie temporal % positividad en Honduras

In [8]:
## Dar Formato al campo Fecha Resultado y crear nueva columna tipo Fecha Corta
df_covid['Fecha'] = df_covid["FechaResultado"].astype(str).str[:10]

# paso1. crear una tabla resumen agrupando los casos por Fecha
df_SerieTemp = df_covid.groupby(['Fecha','Resultado']).size().reset_index(name='casos')

# paso2. crear una tabla pivote y crear las columnas calculadas
df_SerieTemp = df_SerieTemp.pivot(index="Fecha",columns="Resultado", values="casos")
df_SerieTemp['Total_Muestras'] = df_SerieTemp['POSITIVO'] + df_SerieTemp['NEGATIVO'] 
df_SerieTemp['Positividad%'] = (df_SerieTemp['POSITIVO'] / df_SerieTemp['Total_Muestras'])#.apply('{:.0%}'.format)
# quitar comentario de formato % para la tabla, para graficar dejar comentado

# paso3. Convertir la tabla pivote a tabla y seleccionar las columnas para graficar la serie temporal
df_Tabla = df_SerieTemp.reset_index().fillna(0)
df_Tabla = df_Tabla[["Fecha","NEGATIVO","POSITIVO","Total_Muestras","Positividad%"]]

# paso4. Crear la Grafica
import numpy as np

x0 = df_Tabla["Fecha"].iloc[0]            # La primera
x1 = df_Tabla["Fecha"].iloc[-1]           # La última
xm = df_Tabla["Fecha"].iloc[len(df_Tabla)//2]   # La del medio
mean_positividad = np.mean(df_Tabla["Positividad%"])


data = [go.Line(x=df_Tabla["Fecha"],y=df_Tabla["Positividad%"],marker={"color":"#38BCB0"})]
layout = go.Layout(title="% Positividad COVID19 por Año en Honduras", font=dict(size=18),
                   yaxis=dict(title=" % de Positividad", tickformat=".0%"),
                   xaxis=dict(title="FECHA (Mes / Año)")                   
                   )

fig = go.Figure(data=data, layout=layout)

fig.add_shape(type="line",
    x0=x0, y0=mean_positividad, x1=x1, y1=mean_positividad,
    line=dict(color="Red",width=3)
 )

fig.add_annotation(x=xm, y=mean_positividad,
            text='Promedio Nacional de Positividad',
            showarrow=False,
            yshift=10)

pyo.plot(fig,filename="../resultados/Line_Plot_PositividadHNByFecha.html")

#df_Tabla
df_Tabla.to_excel("../resultados/df_Tabla.xlsx")

#fig= px.line(df_Tabla, x='Fecha', y='Positividad%')
#fig.show()






C:\Users\wsilva\AppData\Local\Temp\ipykernel_19044\2303171198.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\wsilva\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




### Crear Tabla de Incidencia de Casos y % de Positividad x Departamento en Honduras

In [5]:
df_poblacionByDepto = pd.read_excel("../dataset/PoblacionHNByDepto.xlsx")
df_poblacionByDepto['poblacion']= df_poblacionByDepto['Urbano Año 2021']+df_poblacionByDepto['Rural Año 2021']

var_anio = 2021

#print(df_covid.columns)
# paso1. crear una tabla resumen agrupando los casos por departamento y filtrando por año
#df_TotalesByDepto = df_covid[df_covid['año']].groupby(['Departamento','Resultado']).size().reset_index(name='casos')
df_TotalesByDepto = df_covid.groupby(['Departamento','Resultado']).size().reset_index(name='casos')
df_TotalesByDepto = df_TotalesByDepto.pivot(index="Departamento",columns="Resultado", values="casos")

# paso2 . crear inner join con tabla de poblacion por departamento
# Crear nuevas columnas calculando el % de poblacion y tasa de incidencia
# inner join con tabla de poblacion
df_TotalesByDepto = pd.merge(df_TotalesByDepto,df_poblacionByDepto, on='Departamento',how='inner')
df_TotalesByDepto['Tasa_100mil'] = (round((df_TotalesByDepto['POSITIVO'] / df_TotalesByDepto['poblacion']) * 100000)).astype('int')
df_TotalesByDepto['Total_Muestras'] = df_TotalesByDepto['POSITIVO'] + df_TotalesByDepto['NEGATIVO']
df_TotalesByDepto['Positividad%'] = (df_TotalesByDepto['POSITIVO'] / df_TotalesByDepto['Total_Muestras'])#.apply('{:.0%}'.format)
# quitar comentario de formato % para la tabla, para graficar dejar comentado

# Seleccion de Columnas
df_mapa = df_TotalesByDepto[["Departamento","Latitud","Longitud","Tasa_100mil"]]
df_TotalesByDepto = df_TotalesByDepto[["Departamento","poblacion","NEGATIVO","POSITIVO","Total_Muestras","Positividad%","Tasa_100mil"]]

df_TotalesByDepto
df_mapa
## Exportar Data Frame
df_TotalesByDepto.to_excel("../resultados/df_TotalesByDepto.xlsx")
df_mapa.to_excel("../resultados/df_mapa.xlsx")
#df_poblacionByDepto

### Grafico_4 Incidencia de Casos de COVID19 por Departamento 

In [313]:
## Crear grafico de barras con plotly  para visualizar la incidencia de casos por Departamento
## tasa calculada por 100,000 habitantes
## Tasa Incidencia = (Positivos / Poblacion) x 100 mil habitantes

import plotly.offline as pyo 
import plotly.express as px
import plotly.graph_objs as go 
import numpy as np

## Ordenar el Data Frame
df_TotalesByDepto = df_TotalesByDepto.sort_values(by="Tasa_100mil",ascending=False)


x0 = df_TotalesByDepto["Departamento"].iloc[0]            # La primera
x1 = df_TotalesByDepto["Departamento"].iloc[-1]           # La última
xm = df_TotalesByDepto["Departamento"].iloc[len(df_TotalesByDepto)//2]   # La del medio
mean_tasa = np.mean(df_TotalesByDepto["Tasa_100mil"])



data = [go.Bar(x=df_TotalesByDepto["Departamento"],y=df_TotalesByDepto["Tasa_100mil"],marker={"color":"#2E7DA1"})]
layout = go.Layout(title="Incidencia de Casos de COVID19 por Departamento", font=dict(size=18),
                   yaxis=dict(title="Tasa x 100,000 Habitantes"),
                   xaxis=dict(title="Departamentos")                   
                   )

fig = go.Figure(data=data, layout=layout)


fig.add_shape(type="line",
    x0=x0, y0=mean_tasa, x1=x1, y1=mean_tasa,
    line=dict(color="Red",width=3)
 )

fig.add_annotation(x=xm, y=mean_tasa,
            text='Tasa Promedio Nacional',
            showarrow=False,
            yshift=10)

pyo.plot(fig,filename="../resultados/Bar_Plot_IncidenciaByDepto.html")
### Codigo para libreria plotly express
# fig1 = px.bar(df_Totales, x="año", y="tasa", color="año")
# fig1.show()

'../resultados/Bar_Plot_IncidenciaByDepto.html'

### Creacion de Mapa Segun la Incidencia de Covid en Honduras

In [318]:
#Importación de librerías
import plotly.offline as pyo
import plotly.graph_objs as go
import pandas as pd

#Carga de datos
#df_mapa 

#Definimos la cadena con el token de la cuenta de mapbox
mapbox_access_token = "pk.eyJ1Ijoid25zZyIsImEiOiJjbGliOTQ2NGwwOWw0M2V0NzVveWc0OWRoIn0.ENqvqKKlrZerjSufywTKfg"#open(".mapbox_token").read()

#Creamos directamente la figura, obviamos data puesto que directamente lo definimos como el primer argumento de Figure
fig = go.Figure(go.Scattermapbox(
        lon = df_mapa['Longitud'], #Seleccionamos nuestra columna con la coordenada longitud
        lat = df_mapa['Latitud'], #Seleccionamos nuestra columna con la coordenada latitud
        mode='markers', text=df_mapa['Tasa_100mil'],
        marker=go.scattermapbox.Marker(
            size=df_mapa["Tasa_100mil"]/40, #Indicamos que el tamaño dependa del tráfico promedio de peatones y escalamos entre 10 para no inundar el mapa
            color=df_mapa["Tasa_100mil"] #Indicamos que el color dependa del tráfico promedio de peatones (podemos usar otra variable)
        )
    ))

#Actualizamos la propiedad "layout" de la figura
fig.update_layout(
    autosize=True,
    hovermode='closest', #muestra el dato más cercano al mover el cursor por el gráfico
    mapbox=dict(
        accesstoken=mapbox_access_token,
        center=dict(
            lat=14.754, #coordenadas del centro de nuestro mapa inicial
            lon=-86.429 #coordenadas del centro de nuestro mapa inicial
        ),
        zoom=7 #zoom inicial del mapa
    ),
)

pyo.plot(fig, filename="../resultados/Geomap_IncidenciaCovidHN.html")

'../resultados/Geomap_IncidenciaCovidHN.html'

### Creacion de tabla  para comparar la incidencia de casos covid entre los 2 departamentos con mayor poblacion en Honduras

In [327]:
#df_poblacionByDepto = pd.read_excel("../dataset/PoblacionHNByDepto.xlsx")
#df_poblacionByDepto['poblacion']= df_poblacionByDepto['Urbano Año 2021']+df_poblacionByDepto['Rural Año 2021']

var_anio = 2021
var_Depto1 ="FRANCISCO MORAZAN"
var_Depto2 ="CORTES"
var_pobDepto1 = 1699753
var_pobDepto2 = 1818981

## crear condiciones para filtrar datos para pruebas de COVID
condicion1 = df_covid.Departamento.isin([var_Depto1,var_Depto2])
condicion2 = df_covid["año"]==2021
condicion3 = df_covid["Resultado"]=="POSITIVO"

## ejecutar filtro de datos
df_covid_Deptos = df_covid[condicion1 & condicion2 & condicion3]


#print(df_covid.columns)
# paso1. crear una tabla resumen agrupando los casos por departamento y filtrando por año
df_RelDeptos = df_covid_Deptos.groupby(['Fecha','Departamento'], as_index=False).agg({'Resultado':'count'})
df_RelDeptos = df_RelDeptos.pivot(index="Fecha",columns="Departamento", values="Resultado")

# paso3. Convertir la tabla pivote a tabla y seleccionar las columnas para graficar la serie temporal
df_RelDeptos = df_RelDeptos.reset_index().fillna(0)
df_RelDeptos['tasa_Cortes'] = (round((df_RelDeptos['CORTES'] / var_pobDepto2) * 100000)).astype('int')
df_RelDeptos['tasa_FM'] = (round((df_RelDeptos['FRANCISCO MORAZAN'] / var_pobDepto1) * 100000)).astype('int')




df_RelDeptos


Departamento,Fecha,CORTES,FRANCISCO MORAZAN,tasa_Cortes,tasa_FM
0,2021-01-01,0.0,54.0,0,3
1,2021-01-02,0.0,55.0,0,3
2,2021-01-03,0.0,66.0,0,4
3,2021-01-04,0.0,43.0,0,3
4,2021-01-05,0.0,103.0,0,6
...,...,...,...,...,...
360,2021-12-27,28.0,44.0,2,3
361,2021-12-28,18.0,70.0,1,4
362,2021-12-29,12.0,79.0,1,5
363,2021-12-30,35.0,109.0,2,6


### Grafica para comparar la incidencia de casos entre Francisco Morazan y Cortes 
### Departamentos con mayor poblacion en Honduras.

In [328]:

#Carga de datos
#df_RelDeptos

#Definición de objeto de tipo lista "data": datos utilizados en el gráfico y el tipo de gráfico. Uso de diferentes trazas
trace0 = go.Scatter(x=df_RelDeptos["Fecha"],
                    y=df_RelDeptos["tasa_Cortes"],
                    mode="lines",
                    name="CORTES")

trace1 = go.Scatter(x=df_RelDeptos["Fecha"],
                    y=df_RelDeptos["tasa_FM"],
                    mode="lines",
                    name="FRANCISCO MORAZAN")

data = [trace0, trace1]

#Definición de objeto "layout": diseño del gráfico como título, nombres de ejes,...
layout = go.Layout(title="Comparacion de incidencia de casos COVID Deptos.(CORTES vs FM) Año 2021",
                    xaxis=dict(title="(Mes / Año)"),
                    yaxis=dict(title="Incidencia x 100,000 Habitantes"))

#Creación de objeto "Figure" de Plotly a partir de los objetos data y layout creados previamente
fig = go.Figure(data=data, layout=layout)
#Generación del plot a partir de la figura definida y nombre del fichero de salida HTML
pyo.plot(fig, filename="../resultados/Line_plot_Incidencia_Cortes_FM.html")

'../resultados/Line_plot_Incidencia_Cortes_FM.html'

### Caracterizacion por sexo y grupo de edad para crear grafico de piramide poblacional con proporcion de positividad por sexo

In [9]:
## Crear columna rango de edad
df_covid['RangoEdad'] = pd.cut(x=df_covid['Edad'],bins=[1,15,25,35,45,55,65,75,np.inf],
                               labels=['< 15', '15 - 24','25 - 34','35 - 44','45 - 54','55 - 64','65 - 74','>= 75'])

## Filtrar solo positivos
in_positivos = df_covid['Resultado']=="POSITIVO"
df_rango_edad = df_covid[in_positivos]


# paso1. crear una tabla resumen agrupando los casos por rango de Edad
df_GroupByEdades = df_rango_edad.groupby(['RangoEdad','Sexo']).size().reset_index(name='positivos')
df_GroupByEdades = df_GroupByEdades.pivot(index="RangoEdad",columns="Sexo", values="positivos")
# paso3. Convertir la tabla pivote a tabla y seleccionar las columnas para graficar la serie temporal
df_GroupByEdades = df_GroupByEdades.reset_index()

df_GroupByEdades['-MUJER'] = df_GroupByEdades['MUJER'] *(-1)
total_H = df_GroupByEdades['HOMBRE'].sum()
total_M = df_GroupByEdades['MUJER'].sum()

df_GroupByEdades['%H'] = (df_GroupByEdades['HOMBRE']/ total_H).apply('{:.0%}'.format)
df_GroupByEdades['%M'] = ((df_GroupByEdades['MUJER']/ total_M)).apply('{:.0%}'.format)
df_GroupByEdades

df_GroupByEdades.to_excel("../resultados/df_GroupByEdades.xlsx")



C:\Users\wsilva\AppData\Local\Temp\ipykernel_19044\901277595.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Grafico Piramide Poblacional

In [379]:
#import plotly.plotly as py
#import plotly.graph_objs as go

#import numpy as np

women_bins = df_GroupByEdades['-MUJER']
men_bins = df_GroupByEdades['HOMBRE']


y = df_GroupByEdades['RangoEdad']

layout = go.Layout(yaxis=go.layout.YAxis(title='Grupos de Edad'),
                   xaxis=go.layout.XAxis(
                       range=[-50000, 50000],
                       tickvals=[-50000, -25000, -10000, 0, 10000, 25000, 50000],
                       ticktext=[50000, 25000, 10000, 0, 10000, 25000, 50000],
                       title='TOTAL'),
                   barmode='overlay',
                   bargap=0.1)

data = [go.Bar(y=y,
               x=men_bins,
               orientation='h',
               name='HOMBRES',
               text=men_bins.astype('int'),
               hoverinfo='x',
               marker=dict(color='#2E7DA1')
               ),
        go.Bar(y=y,
               x=women_bins,
               orientation='h',
               name='MUJERES',
               text=-1 * women_bins.astype('int'),
               hoverinfo='text',
               marker=dict(color='#CD6155')
               )]

pyo.iplot(dict(data=data, layout=layout), filename='../resultados/PiramidePoblacion.html')